In [3]:
import ipywidgets as widgets
from IPython.display import display
import warnings
import PyPDF20
import datetime
import functools

warnings.filterwarnings('ignore')

# Create the file upload widget
file_upload = widgets.FileUpload(
    accept='',  # Accepted file types (e.g., '.txt', '.pdf', '.jpg', '.csv', etc.); leave empty for all file types
    multiple=True  # Set to True if you want to allow multiple files to be uploaded
)

fileListOutput = widgets.Output()

# Display the widget
display(file_upload, fileListOutput)

ongFileUploadVals = []

def on_value_change(change):
    global ongFileUploadVals
    global file_upload
    
    if (ongFileUploadVals ==  [] and list(file_upload.value) == []) or ongFileUploadVals != list(file_upload.value):

        @fileListOutput.capture()
        def file_list_output_render():
            global ongFileUploadVals
            
            fileListOutput.clear_output()
            display(widgets.Label("Uploaded Files: "))
            
            if change != {}:

                # set file_upload to previous file_upload and current file_upload files
                updatedFileUploadVal = []
                for file in ongFileUploadVals:
                    updatedFileUploadVal.append(file)

                for file in file_upload.value:
                    if file not in updatedFileUploadVal:
                        updatedFileUploadVal.append(file)

                # update ong_file_upload value
                ongFileUploadVals = updatedFileUploadVal
            
                for file in ongFileUploadVals:
                    # extract number of pages of uploaded files
                    
                    fileOpen = open(file['name'], 'rb')
                    reader = PyPDF2.PdfReader(fileOpen)
                    totalPgs = len(reader.pages)
                    remButton = widgets.Button(description='Remove')
                    
                    def remove_button_onclick(button, fileInfo=""):
                        global ongFileUploadVals
                        # update file_upload.value to remove chosen file
                        newVal = []
                        for file in file_upload.value:
                            if file['name'] != fileInfo:
                                newVal.append(file)
                        ongFileUploadVals = []
                        file_upload.value = tuple(newVal)
                        
                    remButton.on_click(functools.partial(remove_button_onclick, fileInfo=file['name']))
                    
                    display(widgets.Label("\n  file name: " + file['name'] + "  #pages: " + str(totalPgs)), remButton)

                    file_upload.value = tuple(updatedFileUploadVal)

        file_list_output_render()

file_upload.observe(on_value_change, names='value')

FileUpload(value=(), description='Upload', multiple=True)

Output()